In [ ]:
from networkit import *
import math, sys, time, subprocess, os, numpy

In [ ]:
kaHipAvailable = True
kaHipPath = "/home/moritzl/Gadgets/KaHIP/deploy/kaffpa"

In [ ]:
emptylist = {}

In [ ]:
graphlist = {"bacteriorhodopsin-10-10", "bacteriorhodopsin-10-2.5", "bacteriorhodopsin-10-5", "bacteriorhodopsin", "br_corr_2", "bubble", "ubiquitin"}

In [ ]:
klist = [2**i for i in range(2,7)]

In [ ]:
imbalance = 0.3

In [ ]:
G = readGraph('../../input/ubiquitin.graph', Format.METIS)
n = G.numberOfNodes()
k = 32
X = round(n/k)
tolerance = round(imbalance*round(n/k))
part = continuousPartition(G, X, tolerance, k)

In [ ]:
X

In [ ]:
def partitionValid(G, partition, maxBlockSize = 0, isCharged = []):
    z = G.upperNodeIdBound()
    n = G.numberOfNodes()
    if len(partition) != z:
        return False
    
    if len(isCharged) != 0 and len(isCharged) != z:
        return False
    
    if len(isCharged) == 0:
        isCharged = [False for i in range(z)]
        
    if maxBlockSize == 0:
        maxBlockSize = n
        
    chargedFragments = set()
    
    fragmentSizes = {}
    
    for v in range(G.numberOfNodes()):
        if not G.hasNode(v):
            print("Node ", v, " not in graph.")
            return False
            
        # partition invalid if two charged nodes in same fragment
        if isCharged[v]:
            if partition[v] in chargedFragments:
                print("Node", v, " is charged, but fragment", partition[v], "already has a charged node.")
                return False
            else:
                chargedFragments.add(partition[v])
    
        # partition also invalid if gaps of size 1 exist
        if G.hasNode(v+2) and partition[v+2] == partition[v] and G.hasNode(v+1) and partition[v+1] != partition[v]:
            print("Nodes", v, "and", v+2, "are in fragment", partition[v], "but", v+1, "is in fragment", partition[v+1])
            return False
    
        # partition invalid if fragment is larger than allowed
        if not partition[v] in fragmentSizes:
            fragmentSizes[partition[v]] = 1
        else:
            fragmentSizes[partition[v]] += 1
        if fragmentSizes[partition[v]] > maxBlockSize:
            print("Fragment", partition[v], "contains", fragmentSizes[partition[v]], "nodes, more than", maxBlockSize)
            return False
    
    # no reason to complain found, partition is valid
    return True

In [ ]:
def repairPartition(G, partition, maxBlockSize = 0, isCharged = []):
    """
    Repairs errors in partition, for example multiple charged nodes in the same partition or gaps of size 1
    """
    z = G.upperNodeIdBound()
    assert(len(partition) == z)
    assert(len(isCharged) == 0 or len(isCharged) == z)
    if len(isCharged) == 0:
        isCharged = [False for i in range(z)]
    
    fragmentSet = {partition[v] for v in range(len(partition))}
    
    def findBestMove(u):
        allowedMoves = [f for f in fragmentSet if moveAllowed(G, partition, u, f, maxBlockSize, isCharged)]
        weights = getCutWeights(G, partition, u)
        if len(allowedMoves) == 0:
            print("No allowed target partition found for node", u, ", creating new partition.")
            newFragmentID= max(fragmentSet)+1
            fragmentSet.add(newFragmentID)
            return newFragmentID

        assert(len(allowedMoves) > 0)
        bestMove = allowedMoves[0]
        for move in allowedMoves:
            if weights[move] > weights[bestMove]:
                #we want to minimize the total weight of cut edges.
                #Thus we select as target partition the one with the highest edge weight to the current node
                bestMove = move
        return bestMove
    
    for u in G.nodes():
        #can the node stay where it is?
        if not moveAllowed(G, partition, u, partition[u], maxBlockSize, isCharged):
            # if not, find the best move for it
            partition[u] = findBestMove(u)
    return partition

In [ ]:
def getCutWeights(G, part, v):
    """
    Returns a dictionary with block numbers as keys and cut weights as values
    """
    n = G.numberOfNodes()
    assert(len(part) == n)
    assert(v < n)
    
    sums = {}
    
    for block in set(part):
        sums[block] = 0
        
    for u in G.neighbors(v):
        sums[part[v]] += G.weight(u,v)
        
    return sums

In [ ]:
def moveAllowed(G, partition, v, targetBlock, maxBlockSize = 0, isCharged = []):
    """
    Returns False/True, depending on whether it is allowed to move node v to block targetBlock without violating the charge and gap constraints
    """
    
    z = G.upperNodeIdBound()
    n = G.numberOfNodes()
    assert(n <= z)
    assert(len(partition) == z)
    assert(G.hasNode(v))
    assert(len(isCharged) == 0 or len(isCharged) == z)
    
    if maxBlockSize == 0:
        maxBlockSize = n
        
    targetBlockSize = sum([partition[u] == targetBlock for u in G.nodes()])
    
    if len(isCharged) == 0:
        isCharged = [False for i in range(z)]
    
    # move would be forbidden if node and target partition are already charged
    if isCharged[v] and any([isCharged[u] for u in G.nodes() if partition[u] == targetBlock and u != v]):
        return False
    
    # move would also be forbidden if gap in main chain of size 1 is created
    if v-2 >= 0 and G.hasNode(v-2) and partition[v-2] == targetBlock and G.hasNode(v-1) and partition[v-1] != targetBlock:
        return False
    
    if G.hasNode(v+2) and partition[v+2] == targetBlock and G.hasNode(v+1) and partition[v+1] != targetBlock:
        return False
    
    # move is forbidden if targetBlock is already at maximum capacity
    if targetBlockSize > maxBlockSize or (targetBlockSize == maxBlockSize and partition[v] != targetBlock):
        return False
    
    # move is allowed!
    return True
    

In [ ]:
def continuousPartition(G, X, tolerance, k=0, isCharged={}):
    # validate input
    n = G.numberOfNodes()
    if len(isCharged) > 0:
        assert(len(isCharged)==n)
        if k > 0:
            assert(sum(isCharged) <= k)
    else:
        isCharged = [False for i in range(n)]
    assert(X > 0)
    assert(tolerance >= 0)
    if (n % X > 0 or (k>0 and k*X != n)) and tolerance == 0:
        if (k > 0):
            print("Creating", k, "partitions of size", X, "from a graph with", n, " nodes and tolerance 0 is impossible.")
        else:
            print("Creating partitions of size", X, "from a graph with", n, " nodes and tolerance 0 is impossible.")
        print("Set tolerance to 1.")
        tolerance = 1
        
    if k > 0:
        assert(n <= (X + tolerance)*k)
        assert(n >= (X - tolerance)*k)
        
    maxNumberOfPartitions = int(n / max(X - tolerance, 1)) if (k == 0) else k
    
    def w(i,l):
        """
        Weight of cutting after node i.
        """
        assert(i >= 0)
        assert(i < n)
        weightSum = 0
        for k in range(l+1,i+1):
            for neighbor in G.neighbors(k):
                if neighbor <= l or neighbor > i:
                    weightSum += G.weight(k, neighbor)
        return weightSum
    
    # allocate cut and predecessor table
    table = [[float("inf") for j in range(maxNumberOfPartitions)] for i in range(n)]
    pred = [[-1 for j in range(maxNumberOfPartitions)] for i in range(n)]
    
    # fill table 
    for i in range(n):
        for j in range(maxNumberOfPartitions):
            if (j == 0):
                if abs(i+1-X) <= tolerance:
                    table[i][j] = w(i,-1)
            elif (i >= (X-tolerance)):
                windowStart = max(i-(X+tolerance),0)
                
                # make sure that no two charged nodes are in the same partition
                chargeEncountered = False
                for l in reversed(range(windowStart, i+1)):
                    assert(l >= windowStart)
                    if isCharged[l]:
                        if chargeEncountered:
                            windowStart = l
                            break
                        else:
                            chargeEncountered = True
                
                predList = [table[l][j-1] + w(i,l) for l in range(windowStart, min(i,i-(X-tolerance)+1))]
                if (len(predList) > 0):
                    minPred = min(predList)
                    table[i][j] = minPred
                    pred[i][j] = predList.index(minPred) + windowStart
                    
    #for i in range(n):
    #    row = table[i]
    #    print(i, ': [',end="")
    #    for j in range(len(row)):
    #        elem = row[j]
    #        if math.isinf(elem):
    #            print(", ", end="")
    #        else:
    #            if j > 0:
    #                assert(pred[i][j] >= 0)
    #            print(int(elem), ",", end="", sep="")
    #    print("]")
                    
    # get result from table
    resultlist = [table[n-1][j] for j in range(maxNumberOfPartitions)]
    if len(resultlist) == 0:
        raise ValueError("Combination of parameters allows no partition!")
    
    # if k is given, select best solution with k subsets. If not, select best overall solution
    if (k == 0):
        bestCutValue = min(resultlist)
        k = resultlist.index(bestCutValue) + 1
    else:
        bestCutValue = table[n-1][k-1]
        
    if (bestCutValue == float("inf")):
        raise ValueError("Combination of parameters allows no partition!")
    result = partitioning.Partition(n)
    result.setUpperBound(k)
    
    # search best path backwards
    j = k-1
    i = n-1
    c = bestCutValue
    
    while (j > 0):
        nextI = pred[i][j]
        assert(nextI >= 0)
        # assign partitions to nodes
        for l in range(nextI+1, i+1):
            result[l] = j
        j -= 1
        c -=w(i,nextI)
        i = nextI
        
    # assign partitions to first nodes not covered by previous loop
    for l in range(0, nextI+1):
        result[l] = 0
        
    # check results:
    for i in range(n):
        assert(result[i] >= 0)
        assert(result[i] < k)
        
    for size in result.subsetSizes():
        if (abs(size-X) > tolerance):
            print("For n=", n, ", k=", k, ", X=", X, ", tolerance=", tolerance, ", ", size, " is wrong.")
        assert(abs(size-X) <= tolerance)
    
    return result

In [ ]:
def readInDistances(path):
    # read distances into array
    f = open(path, 'r')
    dd = numpy.fromfile(f, dtype=float, count=-1, sep=' ')
    f.close()

    # first number in file is number of residues
    num_res = int(dd[0])

    dists = numpy.zeros((num_res,num_res))
    dist_to_mol2 = numpy.zeros(num_res)

    dd_line = num_res+1

    # write distances into distance matrix
    for i in range(num_res):
        dist_to_mol2[i]=dd[i+1]
        for j in range(num_res-i):
            dists[i,j+i] = dd[dd_line+j]
            dists[j+i,i] = dd[dd_line+j]
        dd_line = dd_line+num_res-i
        
    return dists, dist_to_mol2

In [ ]:
def qmError(part, dists, dist_to_mol2):
    """
    Calculates the quantum mechanical error induced by this partition
    Parameters
    -----------
    part - partition
    dists - distances between any pair of vertices
    dist_to_mol2 - distance to point of interest
    """
    
    strong = 5.0
    weak = 10.0
    n = len(part)
    sum_edges = 0

    
    assert(len(dists) == n)
    assert(len(dist_to_mol2) == n)
    errorSum = 0
    for i in range(n):
        for j in range(i+1, n):
            if (j == i+1) : # these are the strong bonds between the residues along the chain
                w = strong/numpy.power((dist_to_mol2[i] + dist_to_mol2[j])/2,3) 

                if (part[i] != part[j]) :
                    sum_edges+=w

            elif (j == i) : # no edge with itself
                pass
            elif dists[i,j] < 2.5 : # these are the weak bonds between residues within a certain distance cutoff
                w = weak/numpy.power((dist_to_mol2[i] + dist_to_mol2[j])/2,3)
                if (part[i] != part[j]) :
                    sum_edges+=w
    return sum_edges

In [ ]:
def getKaHiPPartition(G, k, imbalance=0.03):
    maxWeight = max([G.weight(u,v) for (u,v) in G.edges()])
    
    """
    KaHiP only accepts integer weights, thus we scale and round them.
    Weights must be under 1 million, otherwise the METIS graph writer switches to scientific notation,
    which confuses KaHiP
    """
    scalingFactor = int((10**6-1)/maxWeight)
    
    G2 = Graph(G.numberOfNodes(), True)

    for edge in G.edges():
        G2.addEdge(edge[0], edge[1], int(G.weight(edge[0], edge[1])*scalingFactor))

    tempfilename = "scaled.graph"
    kargument = "--k="+str(k)
    iargument = "--imbalance="+str(imbalance)
    pargument = "--preconfiguration=strong"
    writeGraph(G2, tempfilename, Format.METIS)
    subprocess.call([kaHipPath, tempfilename, kargument, iargument, pargument])
    temppartname = "tmppartition"+str(k)
    part = community.PartitionReader().read(temppartname)
    subprocess.call(["rm", tempfilename, temppartname])
    return part

In [ ]:
def geometricMean(series):
    n = len(series)
    prod = 1
    for elem in series:
        prod *= elem
    return prod**(1/n)

In [ ]:
sumMLP = 0
sumNaive = 0
sumKaHiP = 0
sumDynCont = 0
sumNaiveBetterThanMLP = 0
sumTimeMLP = 0
sumTimeNaive = 0
sumTimeKaHip = 0
sumTimeDyn = 0
ratioList = []
numberOfRuns = len(klist)*len(graphlist)
#dists = numpy.zeros((num_res,num_res))
#dist_to_mol2 = numpy.zeros(num_res)

for filename in graphlist:
    G = readGraph('../../input/'+filename+'.graph', Format.METIS)
    n = G.numberOfNodes()
    total = G.totalEdgeWeight()
    
    print("Graph "+filename,", ", n, " nodes.", flush=True)
    distancesAvailable = False
    if (filename == "ubiquitin"):
        dists, dist_to_mol2 = readInDistances("ubi_dists.dat")
        distancesAvailable = True
    elif (filename == "bubble"): 
        dists, dist_to_mol2 = readInDistances("bubble_dists.dat")
        distancesAvailable = True
    
    for k in klist:
        best = -1
        bestCut = total
        print("k: "+str(k))
        
        #
        # MultiLevelPartitioner
        #
        mlp = partitioning.MultiLevelPartitioner(G, k, imbalance, False, emptylist, True)
        start = time.time()
        mlp.run()
        end = time.time()
        part = mlp.getPartition()
        mlpCut = partitioning.computeEdgeCut(part, G)
        mlpImbalance = partitioning.computeImbalance(part, G)
        print("MLP")
        print("Edge cut:"+str(mlpCut))
        print("Imbalance: "+str(mlpImbalance), flush=True)
        if (distancesAvailable):
            print("QM_Error: "+str(qmError(part, dists, dist_to_mol2)))
        #if (mlpImbalance <= 1+imbalance):
        #    best = 0
        #    bestCut = mlpCut
        sumTimeMLP += end - start
        
        #
        # Naive Partitioning
        #
        start = time.time()
        naivePart = community.ClusteringGenerator().makeContinuousBalancedClustering(G, k)
        end = time.time()
        naiveCut = partitioning.computeEdgeCut(naivePart, G)
        naiveImbalance = partitioning.computeImbalance(naivePart, G)
        print("Naive")
        print("Edge cut:"+str(naiveCut))
        print("Imbalance: "+str(naiveImbalance), flush=True)
        if (distancesAvailable):
            print("QM_Error: "+str(qmError(naivePart, dists, dist_to_mol2)))
        if (naiveImbalance <= 1+imbalance and naiveCut < bestCut):
            best = 1
            bestCut = naiveCut
            sumNaiveBetterThanMLP += 1
            #print("MLP worse than naive. Sad.")
        sumTimeNaive += end - start
        
        #
        # Continuous Partitioning, built with dynamic programming
        #
        start = time.time()
        cont = continuousPartition(G, round(n/k), int(imbalance*round(n/k)), k)
        end = time.time()
        dynCut = partitioning.computeEdgeCut(cont, G)
        assert(dynCut <= naiveCut)
        dynImbalance = partitioning.computeImbalance(cont, G)
        print("DynCont:")
        assert(k==cont.numberOfSubsets())
        print("Edge cut:"+str(dynCut))
        print("Imbalance: "+str(dynImbalance), flush=True)
        if (distancesAvailable):
            print("QM_Error: "+str(qmError(cont, dists, dist_to_mol2)))
        ratioList.append(dynCut / naiveCut)
        if (dynImbalance <= 1+imbalance and dynCut < bestCut):
            best = 3
            bestCut = dynCut
        else:
            print("***********************************************************")
        sumTimeDyn += end - start

        #
        # Calling KaHiP, an external partitioner
        #
        if (kaHipAvailable):
            start = time.time()
            kahipPart = getKaHiPPartition(G, k, imbalance)
            end = time.time()
            if not partitionValid(G, kahipPart, n):
                kahipPart = repairPartition(G, kahipPart, n)
            kahipCut = partitioning.computeEdgeCut(kahipPart, G)
            kahipImbalance = partitioning.computeImbalance(kahipPart, G)
            print("Kaffpa")
            print("Edge cut:"+str(kahipCut))
            print("Imbalance: "+str(kahipImbalance), flush=True)
            if (distancesAvailable):
                print("QM_Error: "+str(qmError(kahipPart, dists, dist_to_mol2)))
            if (kahipImbalance <= 1+imbalance and kahipCut < bestCut):
                best = 2
                bestCut = kahipCut
            sumTimeKaHip += end - start
        
        if (best == 0):
            sumMLP += 1
        elif (best == 1):
            sumNaive += 1
        elif (best == 2):
            sumKaHiP += 1
        elif (best == 3):
            sumDynCont += 1
        else:
            print("No algorithm returned a solution of acceptable balance.")
        
        print("------------------------------------")

print("Score, mlp:", sumMLP, " naive: ", sumNaive, ", KaHiP:", sumKaHiP, ", Cont: ", sumDynCont)
print("In ", sumNaiveBetterThanMLP, " cases, the naive solution was better than the multi-level one.")
print("Average time in seconds, mlp:", sumTimeMLP / numberOfRuns, "naive:", sumTimeNaive / numberOfRuns, "KaHiP:", sumTimeKaHip)
print("Geometric mean of Ratios:", geometricMean(ratioList))